# Part 3: Weakly supervised part-of-speech tagging

_by Andreas Stephan (GitHub: @AndSt, Email: andreas.stephan@univie.ac.at) and Vasiliki Kougia (GitHub: @vasilikikou, Email: vasiliki.kougia@univie.ac.at)_

In this part, we will work on a different type of task, which is called **sequence labeling**. Instead of having one label for an entire text, in sequence labeling, we assign a label to each token in the text.
Specifically we chose **Part-of-speech (POS) tagging**, where the goal is to assign a POS tag that indicates a grammatical type, to a word based on its definition and context.


<img src="../img/pos_tagging.png" width="800" style="display: block; margin: 0 auto" />



In order to perform weakly supervised POS tagging, we will employ the [skweak toolkit](https://github.com/NorskRegnesentral/skweak).
We will create labeling functions to assign POS tags based on _syntactic analysis_ and _grammatical rules_.


In [ ]:
# Imports
%load_ext autoreload
%autoreload 2

import re
import os

import pandas as pd

import nltk
import spacy

from textblob import TextBlob
from textblob.taggers import PatternTagger

import skweak

from scripts.skweak_ner_eval import evaluate
from scripts.utils import load_data_split, get_frequent_words, tag_all, penntreebank2universal, compute_recall, compute_num_conflicts

pd.set_option('display.max_rows', 500)

In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('stopwords')
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_md
!python -m textblob.download_corpora

## POS tags

For this tutorial, we will use the following subset of the [universal POS tags](https://universaldependencies.org/u/pos/index.html):
1. **DET**: determiner, which is a word that modifies nouns or noun phrases and expresses the reference of the noun phrase in context.
2. **NUM**: numeral. It is a word that expresses a number and a relation to the number, such as quantity, sequence, frequency or fraction.
3. **PROPN**: proper noun is a noun that is the name of a specific individual, place, or object.
4. **ADJ**: adjective, which is a word that typically modifies nouns and specifies their properties or attributes.
5. **NOUN**: noun, which is a part of speech typically denoting a person, place, thing, animal or idea.

In [ ]:
all_labels = ["DET", "NUM", "PROPN", "ADJ", "NOUN"]

## Load data

We will use the [English corpus](https://universaldependencies.org/treebanks/en_ewt/index.html) from Universal Dependencies, a framework that contains consistent grammatical annotations across many different languages.
The texts in the corpus come from five types of web media: weblogs, newsgroups, emails, reviews, and Yahoo! answers and consist of 254,825 words and 16,621 sentences.

Skweak operates on spaCy ``doc`` objects, so the dataset is loaded in this format.

In [ ]:
# Load training data
train_docs = load_data_split("train", all_labels)

In [ ]:
for doc in train_docs[:3]:
    skweak.utils.display_entities(doc)

## 3.1 Labeling functions

In the first step, we find the 200 most frequent words in our training corpus and use a lexicon to label these words. In the second step, we mannually annotate the 50 most frequent words.
Finally, for each POS tag we will create the following labeling functions: 

*   DET --> Lexicon with determiners.
*   NUM --> If the token is a number or a word indicating a number from 1 to 10.
*   PROPN --> A word that is capitalized.
*   ADJ --> List of prefixes and suffixes. Syntactic rules that check: 1. if the previous word is a form of "be" and 2. if the previous word is a determiner or numeral.
*   NOUN --> List of prefixes and suffixes. Syntactic rule checking if the previous word is a determiner, numeral or adjective.

#### Lexicon LF

In [ ]:
# Get the 200 most frequent words in the training set
frequent_words = get_frequent_words(train_docs, 200)
print(frequent_words[:5])

In [ ]:
# Load the lexicon
with open("noun_vb_adj_list.txt") as f:
    lines = f.readlines()

# Create a dictionary with the words and their pos tags
lexicon = {}
for l in lines:
    values = l.replace("\n", "").split("\t")
    lexicon[values[0]] = values[1]

In [ ]:
print("There are", len(lexicon), "words in the lexicon.")
print(list(lexicon.items())[:5])

In [ ]:
# How many of the frequent words we found exist in the lexicon
len((list(set(frequent_words) & set(list(lexicon.keys())))))

In [ ]:
# Lexicon LF
def frequent_word_detector(doc):
    for token in doc:
        # If the frequent word exists in the lexicon use its assigned pos tag
        if token.text.lower() in frequent_words and token.text.lower() in list(lexicon.keys()):
            yield token.i, token.i + 1, lexicon[token.text.lower()]


lexicon_lf = skweak.heuristics.FunctionAnnotator("frequent_words", frequent_word_detector)


#### Manual annotation LF

In [ ]:
# Manual annotation
top50_words = get_frequent_words(train_docs, 50)
print(top50_words)

In [ ]:
# Annotate the words that their POS tag exists in our chosen tag subset
manual_tags = {
    "one": "NUM",
    "like": "VERB",
    "time": "NOUN",
    "get": "VERB",
    "know": "VERB",
    "good": "ADJ",
    "could": "VERB",
    "new": "ADJ",
    "go": "VERB",
    "please": "VERB",
    "people": "NOUN",
    "said": "VERB",
    "work": "VERB",
    "bush": "NOUN",
    "want": "VERB",
    "great": "ADJ",
    "way": "NOUN",
    "see": "VERB",
    "best": "ADJ",
    "place": "NOUN",
    "take": "VERB",
    "going": "VERB",
    "service": "NOUN",
    "need": "VERB",
    "make": "VERB",
    "year": "NOUN",
    "number": "NOUN",
    "day": "NOUN",
    "two": "NUM",
    "think": "VERB",
    "food": "NOUN",
    "let": "VERB",
    "first": "ADJ",
    "call": "VERB",
    "2": "NUM",
    "help": "VERB"
}

In [ ]:
# Manual POS tags LF
def manual_pos_tagger(doc):
    for token in doc:
        if token.text.lower() in manual_tags:
            yield token.i, token.i + 1, manual_tags[token.text.lower()]


manual_pos_lf = skweak.heuristics.FunctionAnnotator("manual_pos", manual_pos_tagger)


#### DET LF

In [ ]:
# Use a lexicon of determiners
tries = skweak.gazetteers.extract_json_data("det.json")
det_lf = skweak.gazetteers.GazetteerAnnotator("determiners", tries, case_sensitive=False)


#### NUM LF

In [ ]:
# Use a regular expression pattern to look for digits
def num_detector(doc):
    for token in doc:
        if re.search("\d+", token.text):
            yield token.i, token.i + 1, "NUM"

# Check if the token is the word of a number from 1 to 10
def num_word_detector(doc):
    for token in doc:
        if token.text.lower() in ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten"]:
            yield token.i, token.i + 1, "NUM"

num_lf1 = skweak.heuristics.FunctionAnnotator("numerals1", num_detector)
num_lf2 = skweak.heuristics.FunctionAnnotator("numerals2", num_word_detector)


#### PROPN LF

In [ ]:
# Check if the fist letter of a word or the whole word is capitalized
def propn_detector(doc):
    for token in doc:
        if token.i == 0:
            # For the first word of a sentence, check if all letters are capitalized
            if token.text.isupper():
                yield token.i, token.i + 1, "PROPN"
        else:
            if token.text.isupper() or token.text[0].isupper():
                yield token.i, token.i + 1, "PROPN"


propn_lf = skweak.heuristics.FunctionAnnotator("proper_nouns", propn_detector)


#### ADJ LFs

In [ ]:
# Look for common suffixes and prefixes
def adj_detector_suffixes(doc):
    suffixes = ("able", "al", "ful", "ic", "ive", "less", "ous", "y", "ish", "ible", "ent", "est")
    for token in doc:
        if len(token.text) > 3 and token.text.endswith(suffixes):
            yield token.i, token.i + 1, "ADJ"


# Look for common prefixes
def adj_detector_prefixes(doc):
    prefixes = ("un", "im", "in", "ir", "il", "non", "dis")
    for token in doc:
        if len(token.text) > 3 and token.text.lower().startswith(prefixes):
            yield token.i, token.i + 1, "ADJ"


# If the previous word is a form of "be" and the current word does not end with "ing" and was not labeled as DET, then it's an adjective
def adj_detector_synt1(doc):
    weak_labels = ["O"] * len(doc)
    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            prev = doc[token.i - 1].text.lower()
            if prev in ["be", "been", "being", "am", "is", "are", "was", "were"] and (
                    not token.text.endswith("ing")) and weak_labels[token.i] == "O":
                yield token.i, token.i + 1, "ADJ"


# If the previous word is labeld as DET or NUM, then the current word is an adjective
def adj_detector_synt2(doc):
    weak_labels = ["O"] * len(doc)

    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals1"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals2"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            if weak_labels[token.i - 1] != "O":
                yield token.i, token.i + 1, "ADJ"


adj_lf1 = skweak.heuristics.FunctionAnnotator("adjectives1", adj_detector_suffixes)
adj_lf2 = skweak.heuristics.FunctionAnnotator("adjectives2", adj_detector_prefixes)
adj_lf3 = skweak.heuristics.FunctionAnnotator("adjectives3", adj_detector_synt1)
adj_lf4 = skweak.heuristics.FunctionAnnotator("adjectives4", adj_detector_synt2)


#### NOUN LF

Let's create a labeling function that looks for common noun suffixes. Can you think of some?

In [ ]:
# ***********************************
def noun_detector_suffixes(doc):
    pass
    # TODO
    
# ***********************************

In [ ]:
# Look for common prefixes
def noun_detector_prefixes(doc):
    prefixes = (
        "anti", "auto", "bi", "co", "counter", "dis", "ex", "hyper", "in", "inter", "kilo", "mal", "mega", "mis",
        "mini", "mono", "neo", "out", "poly", "pseudo", "re", "semi", "sub", "super", "sur", "tele", "tri", "ultra",
        "under", "vice")
    for token in doc:
        if len(token.text) > 3 and token.text.lower().startswith(prefixes):
            yield token.i, token.i + 1, "NOUN"


# # If the previous word is labeld as DET, NUM or ADJ, then the current word is an noun
def noun_detector_synt(doc):
    weak_labels = ["O"] * len(doc)

    for span in doc.spans["determiners"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals1"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["numerals2"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives1"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives2"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives3"]:
        weak_labels[span.start] = span.label_

    for span in doc.spans["adjectives4"]:
        weak_labels[span.start] = span.label_

    for token in doc[1:]:
        if not token.is_punct:
            if weak_labels[token.i - 1] != "O":
                yield token.i, token.i + 1, "NOUN"


noun_lf1 = skweak.heuristics.FunctionAnnotator("nouns1", noun_detector_suffixes)
noun_lf2 = skweak.heuristics.FunctionAnnotator("nouns2", noun_detector_prefixes)
noun_lf3 = skweak.heuristics.FunctionAnnotator("nouns3", noun_detector_synt)


## Apply LFs

In [ ]:
# Put all LFs in a list
lfs = [
    lexicon_lf, manual_pos_lf, det_lf, 
    num_lf1, num_lf2, propn_lf,
    adj_lf1, adj_lf2, adj_lf3, adj_lf4,
    noun_lf2, noun_lf3
]

train_docs = tag_all(train_docs, lfs)

In [ ]:
# Print some of the assigned weak labels
for doc in train_docs[:3]:
    skweak.utils.display_entities(doc, ["determiners", "nouns2"])

In [ ]:
# Train HMM
hmm = skweak.aggregation.HMM("hmm", all_labels)
hmm = hmm.fit(train_docs)

In [ ]:
# Majority voting
mv = skweak.aggregation.MajorityVoter("mv", all_labels)

In [ ]:
# Apply LFs, HMM and MV to the test docs
test_docs = load_data_split("test", all_labels)
test_docs = tag_all(test_docs, lfs + [mv, hmm])

## Evaluate

#### Which POS tags are easier to detect?

* We see that POS tags like determiners and numerals are easier to detect and we can achieve a good F1 score with just one or two simple LFs.


In [ ]:
df = evaluate(test_docs, all_labels, [
    "determiners", "numerals1", "numerals2", "proper_nouns"
])

In [ ]:
df.loc[["DET", "NUM", "PROPN"]]

* Other POS tags like adjectives and nouns, which rely more on the context are harder to detect and require more complicated rules.

In [ ]:
df = evaluate(test_docs, all_labels, [
    "adjectives1", "adjectives2", "adjectives3", "adjectives4",
    "nouns1", "nouns2", "nouns3"
])

In [ ]:
df.loc[["ADJ", "NOUN"]]

#### Which type of LF works the best?

* For adjectives the LF that uses suffixes works the best, while the syntactic rules are less accurate. On the contrary, for nouns the LF that is based on syntactic analysis has the best results. For both POS tags, the LFs that use prefixes do not yield good results.

#### Which aggregator works best?

* Despite its simplicity, majority voting outperforms HMM on almost all of the POS tags and overall achieves a higher macro F1 score.

In [ ]:
df = evaluate(test_docs, all_labels, ["mv", "hmm"])

In [ ]:
df

## Using Libraries as Labelling functions


In this part, we use popular NLP libraries to create labeling functions. They include Spacy, NLTK, Textblob.
We use the Majority Voter and HMM as aggregation functions
Optionally, you can train your own model on the data.

Learning goals:
- Understand how to use external libraries as labeling functions
- Understand the Spacy object and how to use it for annotation

First, read and understand the two functions below.

In [ ]:

# Sometimes data formats (here POS tags) differ. We load the data and convert it to the format we need. 
# Surely, there is some loss of information
def nltk_tagger(doc):
    for token in doc:
        if not token.is_punct:
            # Tag token with nltk
            nltk_pos = nltk.pos_tag([token.text])[0][1]
            # Map nltk pos tags to ours
            if nltk_pos == "DT":
                yield token.i, token.i + 1, "DET"
            elif nltk_pos == "CD":
                yield token.i, token.i + 1, "NUM"
            elif nltk_pos == "NNP" or nltk_pos == "NNPS":
                yield token.i, token.i + 1, "PROPN"
            elif nltk_pos == "JJ" or nltk_pos == "JJR" or nltk_pos == "JJS":
                yield token.i, token.i + 1, "ADJ"
            elif nltk_pos == "NN" or nltk_pos == "NNS":
                yield token.i, token.i + 1, "NOUN"
            elif nltk_pos == "VB" or nltk_pos == "VBD" or nltk_pos == "VBG" or nltk_pos == "VBN" or nltk_pos == "VBP" or nltk_pos == "VBZ":
                yield token.i, token.i + 1, "VERB"


# We cn also use the Textblob library to get POS tags
# Under the hood, it uses the Pattern library. Once again, a transformation of the tag-labels is needed
def textblob_tagger(doc):
    for token in doc:
        if not token.is_punct:
            textblob_pos = TextBlob(token.text, pos_tagger=PatternTagger()).tags
            if len(textblob_pos) > 0:
                yield token.i, token.i + 1, penntreebank2universal(textblob_pos[0][1])


## Write the Spacy Labeling Functions

Use the two english Spacy models "en_core_web_sm", "en_core_web_md" to create labeling functions.
The challenge is that they use different tokens, i.e. the atomic units of a sentence. Our simple tokenization just splits the words by whitespace.
Your task it to design an algorithm that maps the tokens of the simple tokenization to the tokens of the Spacy tokenization, and use the token available there to create labeling functions.

Hints:
1) Access token i by `token=doc[i]` or obtain its poition by `i=token.i`
2) Access the Spacy POS token (its ground truth) by `pos=token.pos_`

In [ ]:
eng_nlp_sm = spacy.load("en_core_web_sm")
eng_nlp_md = spacy.load("en_core_web_md")

# ***********************************

def eng_spacy_tagger_sm(doc):
    other_doc = eng_nlp_sm(doc.text)
    # TODO


def eng_spacy_tagger_md(doc):
    pass
    # TODO

# ***********************************

In [ ]:

nltk_lf = skweak.heuristics.FunctionAnnotator("nltk", nltk_tagger)
textblob_lf = skweak.heuristics.FunctionAnnotator("textblob", textblob_tagger)
eng_spacy_sm_lf = skweak.heuristics.FunctionAnnotator("eng_spacy_sm", eng_spacy_tagger_sm)
eng_spacy_md_lf = skweak.heuristics.FunctionAnnotator("eng_spacy_md", eng_spacy_tagger_md)

### Load Data and apply Labeling functions

Before and after applying the labeling functions, and the aggregation functions, we compute the recall and number of conflicts. For the sake of time, we use this time only a subset of the data.

In [ ]:

# load training and test data
lfs = [nltk_lf, eng_spacy_sm_lf, textblob_lf, eng_spacy_md_lf]
all_labels = ["DET", "NUM", "PROPN", "NOUN", "ADJ"]

# small amount of data for the sake of time
train_docs = load_data_split("train", all_labels, 3000)

# tag the training documents
train_docs = tag_all(train_docs, lfs)



In [ ]:
recall = compute_recall(train_docs)
num_conflicts = compute_num_conflicts(train_docs)
print("Train recall", recall)
print("Train conflicts", num_conflicts)

We observe that the recall is very high. This is because the used libraries are quite well.
Further, we observe that in 40.5% of the tokens there is a conflict.

In [ ]:
# train the HMM
hmm = skweak.aggregation.HMM("hmm", all_labels)
hmm=hmm.fit(train_docs)

Now we compare how majority vote and HMM change the number of conflicts.
Remember, that it's important to set Majority vote before HMM, otherwise Majority Vote takes the HMM predictions into account

In [ ]:
mv = skweak.aggregation.MajorityVoter("mv", all_labels)
train_docs = tag_all(train_docs, [mv, hmm])

num_conflicts = compute_num_conflicts(train_docs)
print("Conflicts with MV on train set: ", num_conflicts)

We observe that the number of token conflicts does not change. The reason is that both methods can not choose a class different from the labeling functions.

## Evaluation

Look at the Precision, Recall and F1-Score of the different aggregation functions. What do you observe?

In [ ]:
# tag the test documents
# Once again, it's important to set Majority vote before HMM, otherwise Majority Vote takes the HMM predictions into account
test_docs = load_data_split("test", all_labels, 1000)
test_docs = tag_all(test_docs, lfs + [mv, hmm])

num_conflicts = compute_num_conflicts(test_docs)
print("Conflicts on test set", num_conflicts)

In [ ]:
df = evaluate(test_docs, all_labels, [ "mv", "hmm"])

In [ ]:
df

Contrary, to the first part, we observe that the HMM performs better than majority vote.